In [1]:
import os
import configparser

import openrouteservice
import geopandas as gpd
from shapely.geometry import Point, Polygon

import pandas as pd
#import matplotlib.pyplot as plt
#import matplotlib.patches as mpatches

import requests
import json

#from geopy.geocoders import Nominatim

In [2]:
#set filepaths
project_root = os.path.abspath(os.getcwd())
root = os.path.abspath(os.path.join(os.getcwd(), ".."))
config_file =  os.path.join(root, "config.cfg")

# geodata for shapefiles
config = configparser.ConfigParser()
config.read(config_file)
gdata_root = config["geodata"]["path"]
apikey = config["openroutingservice"]["key"]
base_url = config["openroutingservice"]["base_url"]
client = openrouteservice.Client(key=apikey,base_url=base_url)

In [5]:
#get data from OSM using Overpass API - all the wells for isonchrone collection, we still can use only official well afterwards
# alternative query to olny get wells belonging to the official network, see e.g. https://github.com/technologiestiftung/giessdenkiez-de-osm-pumpen-harvester/blob/master/harvester/fetch.py
# overpass_query = """[out:json];(area["ISO3166-2"="DE-BE"]["admin_level"="4"];)->.searchArea;(node["man_made"="water_well"]["network"="Berliner Straßenbrunnen"](area.searchArea););out;>;out;"""

overpass_url = "http://overpass-api.de/api/interpreter"
overpass_query = """
[out:json];
area[name="Berlin"][admin_level=4]->.searchArea;
(node["man_made"="water_well"](area.searchArea); 
);
out center;
"""
response = requests.get(overpass_url, params={'data': overpass_query})
data = response.json()

filename = os.path.join(gdata_root,'Berlin', 'water_wells_osm.json')
with open(filename, "w") as json_file:
    json.dump(data, json_file, indent=4)

In [6]:
# Extract unique tags from all elements
unique_tags = set()
for element in data['elements']:
    if 'tags' in element:
        unique_tags.update(element['tags'].keys())

unique_tags_list = list(unique_tags)

data_dict = {"id":[], "lat":[], "lon":[]} | {tag: [] for tag in unique_tags_list}

# Populate the data dictionary with values from JSON data
for element in data['elements']:
    data_dict["id"].append(element["id"])
    data_dict["lat"].append(element["lat"])
    data_dict["lon"].append(element["lon"])
    element_tags = element.get('tags', {})
    for tag in unique_tags_list:
        data_dict[tag].append(element_tags.get(tag, None))

# Create a pandas DataFrame from the data dictionary
df = pd.DataFrame(data_dict)
# convert into geodataframe
geometry = [Point(xy) for xy in zip(df['lon'].astype(float), df['lat'].astype(float))]
gdf = gpd.GeoDataFrame(df, crs="EPSG:4326", geometry = geometry)
gdf.to_file(os.path.join(gdata_root,'Berlin', 'water_wells.gpkg'), driver='GPKG')
gdf.head(5)

,id,lat,lon,drinking_water,artist_name,tourism,handle,wheelchair,pump:type,drinking_water:legal,...,mapillary,opening_hours,lda:criteria,image:1,artist:wikidata,wikipedia,description,water_well,operational_status,geometry
0,352734260,52.539807,13.564219,None,None,None,None,None,None,no,...,612227476968597,None,None,None,None,de:Liste der Straßenbrunnen im Berliner Bezirk...,None,None,None,POINT (13.56422 52.53981)
1,359962563,52.536420,13.566712,None,None,None,None,None,None,no,...,768960184438323,None,None,None,None,de:Liste der Straßenbrunnen im Berliner Bezirk...,None,None,None,POINT (13.56671 52.53642)
2,499609652,52.524164,13.308016,no,None,None,None,None,None,no,...,None,None,None,None,None,de:Liste der Straßenbrunnen im Berliner Bezirk...,None,None,None,POINT (13.30802 52.52416)
3,499609653,52.523804,13.311737,no,None,None,None,None,None,no,...,None,None,None,None,None,de:Liste der Straßenbrunnen im Berliner Bezirk...,None,None,None,POINT (13.31174 52.52380)
4,520277232,52.577880,13.404851,no,None,None,None,None,beam_pump,None,...,None,None,None,None,None,None,"Historische, außer Betrieb gesetzte Wasserpumpe",pump,None,POINT (13.40485 52.57788)


In [7]:
gdf[((gdf['emergency']=='drinking_water')|(gdf['drinking_water']=='yes'))].shape[0]

1977

Not all the pumps have 'drinking_water' or 'emergency' tags, but let's assume for the moment that all can be used in case of emergency - can be clarified later.

Now obtain isochrones for each water well (on that stage for all of them and later it will be possible to filter out some). As the ORS (https://openrouteservice.org/) allows only for 500 isochrones calls via API per day, I've spun a docker image of the ORS to calculate them locally. 

In [ ]:
coordinates = list(zip(gdf['lon'].astype(float), gdf['lat'].astype(float)))
ids = list(gdf['id'])
# put it all  in a loop
iso = []
fails=[]

for time in range(60, 3601, 60):
    for idx,location in enumerate(coordinates):
        location = [location]
        #print(location)
        try:
            response = openrouteservice.isochrones.isochrones(client, location, profile='foot-walking',
                                                              range_type='time', range=[time], location_type='start', 
                                                              smoothing=0,attributes=['area'], validate=True)

            data = {"id": [],"center": [], "time": [], "geometry": [] }
            for isochrone in response['features']:
                data["id"].append(ids[idx])
                data["center"].append(isochrone["properties"]["center"])
                data["time"].append(isochrone["properties"]["value"])
                data["geometry"].append(Polygon([list(coord) for coord in isochrone['geometry']['coordinates'][0]]))

            df_single = pd.DataFrame.from_dict(data)
            iso.append(df_single)
        except:
            data = {"id": [] }
            data["id"].append(ids[idx])
            df_single = pd.DataFrame.from_dict(data)
            fails.append(df_single)
iso = pd.concat(iso, axis=0, ignore_index=True)
if len(fails)>0:
    fails = pd.concat(fails, axis=0, ignore_index=True)
#convert isochrones to geo dataframe
iso = gpd.GeoDataFrame(iso, crs ='EPSG:4326', geometry="geometry")

iso["lon"] = iso["center"].str[0]
iso["lat"] = iso["center"].str[1]
iso['time']=iso['time'].astype(int)
iso = iso.drop(columns="center")

# save isochrones to individual layers
for time in iso.time.unique():
    iso[iso['time']==time].to_file(os.path.join(gdata_root,'Berlin', 'water_wells_isochrones3600.gpkg'), driver='GPKG',layer=f"{time:04d}")
if len(fails)>0:
    fails=gpd.GeoDataFrame(fails, crs ='EPSG:4326', geometry="geometry")
    fails.to_file(os.path.join(gdata_root,'Berlin', 'water_wells_isochrones3600_fails.gpkg'), driver='GPKG')